# Exploring various aspects of Austin through multiple data sets

- This analysis explores Austin through using STL decomposition, Time Series data, clustering of high crime zones, mapping incidents using Folium, using an ARIMA model to create an accurate weather forecast, and lastly merging biking and weather data in Austin to explore various trends/patterns.

In [233]:
# Import all necessary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.seasonal import STL
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import StackingRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import FunctionTransformer

In [2]:
# Unfortunately all the data you need to explore a project might not always be easily found in a single data set.
# This project serves as a way for me to learn to collect data in unconvential ways (despite how much I love
# simply downloading a csv from Kaggle) and utilize multiple data sets to gather a wide variety of insights 
# that helps me paint a picture of everything Austin, Texas, which is something I want to explore as a UT Austin 
# student. 

In [3]:
# Sources of Data: datausa.io, data.austintexas.gov, Kaggle

In [4]:
# Race and Ethnicity Data in Austin

ethnicity = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Race and Ethnicity.csv')
ethnicity.head()

,ID Race,Race,ID Ethnicity,Ethnicity,ID Year,Year,Hispanic Population Moe,Geography,ID Geography,Slug Geography,Population,share
0,0,White Alone,0,Not Hispanic or Latino,2020,2020,3882.0,"Austin, TX",16000US4805000,austin-tx,465953,0.482417
1,0,White Alone,1,Hispanic or Latino,2020,2020,5157.0,"Austin, TX",16000US4805000,austin-tx,204290,0.211508
2,1,Black or African American Alone,0,Not Hispanic or Latino,2020,2020,2419.0,"Austin, TX",16000US4805000,austin-tx,71430,0.073954
3,1,Black or African American Alone,1,Hispanic or Latino,2020,2020,912.0,"Austin, TX",16000US4805000,austin-tx,3778,0.003911
4,2,American Indian & Alaska Native Alone,0,Not Hispanic or Latino,2020,2020,347.0,"Austin, TX",16000US4805000,austin-tx,1770,0.001833


In [5]:
# The 3 largest Ethnic Groups in Austin, TX (2016-2020), it is quite evident that White Alone is the prevalent ethnic 
# group yearly, so lets look at largest ethnic groups overall, rather than by year.

# Group the dataframe by Race and Ethnicity and calculate the total population for each group
grouped = ethnicity.groupby(['Race', 'Ethnicity', 'Year'])['Population'].sum().reset_index()

# Sort the groups by population in descending order
sorted_groups = grouped.sort_values('Population', ascending=False)

# Get the top 3 largest ethnic groups
top_3_groups = sorted_groups.head(3)

# Display the top 3 largest ethnic groups
print(top_3_groups[['Race', 'Ethnicity', 'Population', 'Year']])

            Race               Ethnicity  Population  Year
111  White Alone  Not Hispanic or Latino      465953  2020
110  White Alone  Not Hispanic or Latino      459086  2019
109  White Alone  Not Hispanic or Latino      452110  2018


In [6]:
# The 3 largest Ethnic Groups in Austin, TX overall

# Group the dataframe by Race and Ethnicity and calculate the total population for each group
grouped = ethnicity.groupby(['Race', 'Ethnicity'])['Population'].sum().reset_index()

# Sort the groups by population in descending order
sorted_groups = grouped.sort_values('Population', ascending=False)

# Get the top 3 largest ethnic groups
top_3_groups = sorted_groups.head(3)

# Display the top 3 largest ethnic groups
print(top_3_groups[['Race', 'Ethnicity', 'Population']])

                               Race               Ethnicity  Population
13                      White Alone  Not Hispanic or Latino     3529503
12                      White Alone      Hispanic or Latino     1819106
5   Black or African American Alone  Not Hispanic or Latino      538731


In [7]:
universities = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Universities.csv')
universities.head()

,ID Sector,Sector,ID University,University,ID Year,Year,Completions,Slug University,Geography,ID Geography,Slug Geography
0,1,"Public, 4-year or above",222992,Austin Community College District,2020,2020,5590,austin-community-college-district,"Austin, TX",16000US4805000,austin-tx
1,1,"Public, 4-year or above",228778,The University of Texas at Austin,2020,2020,15363,the-university-of-texas-at-austin,"Austin, TX",16000US4805000,austin-tx
2,2,"Private not-for-profit, 4-year or above",225575,Huston-Tillotson University (225575),2020,2020,204,huston-tillotson-university,"Austin, TX",16000US4805000,austin-tx
3,2,"Private not-for-profit, 4-year or above",224004,Concordia University Texas (224004),2020,2020,859,concordia-university-texas,"Austin, TX",16000US4805000,austin-tx
4,2,"Private not-for-profit, 4-year or above",227845,Saint Edward's University,2020,2020,1175,saint-edwards-university,"Austin, TX",16000US4805000,austin-tx


In [8]:
# Showcase universities in Austin, Texas with the most college graduates

top_3_universities = universities.groupby(['University'])['Completions'].sum().reset_index()

# Sort the groups by population in descending order
sorted_universities = top_3_universities.sort_values('Completions', ascending=False)

# Get the top 3 largest ethnic groups
top_3_groups = sorted_universities.head(3)

print(top_3_groups)

                           University  Completions
27  The University of Texas at Austin       139289
5   Austin Community College District        33082
21          Saint Edward's University        10784


In [169]:
# Looking at Austin Crime Data

crime = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/rows.csv')

crime.head()

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_7284/3329683966.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  crime = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/rows.csv')


,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,2019891702,CRIMINAL TRESPASS,2716,N,03/30/2019 08:44:00 PM,03/30/2019,2044.0,03/30/2019 08:59:00 PM,03/30/2019,2059.0,...,11.00,C,03/30/2019,NaN,NaN,3117057.0,3117057.0,30.269060,-97.734085,"(30.26905967, -97.73408544)"
1,20195013370,THEFT,600,N,03/23/2019 06:00:00 PM,03/23/2019,1800.0,04/01/2019 03:47:00 PM,04/01/2019,1547.0,...,21.07,N,04/03/2019,23H,Theft,3133783.0,3133783.0,30.302208,-97.680177,"(30.30220794, -97.6801768)"
2,20172571868,AGG ROBBERY/DEADLY WEAPON,300,N,09/14/2017 10:37:00 PM,09/14/2017,2237.0,09/14/2017 10:37:00 PM,09/14/2017,2237.0,...,18.04,O,09/20/2017,120,Robbery,3125442.0,3125442.0,30.333542,-97.705762,"(30.33354183, -97.70576196)"
3,2019920086,FAMILY DISTURBANCE,3400,N,04/02/2019 01:49:00 AM,04/02/2019,149.0,04/02/2019 01:49:00 AM,04/02/2019,149.0,...,15.03,NaN,NaN,NaN,NaN,3125614.0,3125614.0,30.332481,-97.705246,"(30.3324806, -97.70524551)"
4,2019891685,POSS MARIJUANA,1803,N,03/30/2019 10:36:00 PM,03/30/2019,2236.0,03/30/2019 10:36:00 PM,03/30/2019,2236.0,...,7.00,C,04/02/2019,NaN,NaN,3117721.0,3117721.0,30.274861,-97.731825,"(30.27486103, -97.73182518)"


In [10]:
# Creating clusters based off of locations to look for trends within crime data

from sklearn.cluster import KMeans

crime.dropna(inplace=True)

# create a new dataframe with just the location information
loc_df = crime[["Latitude", "Longitude"]]

# normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
loc_scaled = scaler.fit_transform(loc_df)

# perform k-means clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=42).fit(loc_scaled)

# add cluster labels to original dataframe
crime["location_cluster"] = kmeans.labels_
crime.head()

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location,location_cluster
1,20195013370,THEFT,600,N,03/23/2019 06:00:00 PM,03/23/2019,1800.0,04/01/2019 03:47:00 PM,04/01/2019,1547.0,...,N,04/03/2019,23H,Theft,3133783.0,3133783.0,30.302208,-97.680177,"(30.30220794, -97.6801768)",4
2,20172571868,AGG ROBBERY/DEADLY WEAPON,300,N,09/14/2017 10:37:00 PM,09/14/2017,2237.0,09/14/2017 10:37:00 PM,09/14/2017,2237.0,...,O,09/20/2017,120,Robbery,3125442.0,3125442.0,30.333542,-97.705762,"(30.33354183, -97.70576196)",1
6,20145045267,BURGLARY OF VEHICLE,601,N,10/11/2014 05:00:00 PM,10/11/2014,1700.0,10/12/2014 04:13:00 AM,10/12/2014,413.0,...,N,11/04/2014,23F,Theft,3115529.0,3115529.0,30.268006,-97.738955,"(30.26800598, -97.73895531)",0
12,2019940581,THEFT FROM PERSON,610,N,04/02/2019 06:00:00 PM,04/02/2019,1800.0,04/04/2019 09:40:00 AM,04/04/2019,940.0,...,N,04/05/2019,23A,Theft,3101694.0,3101694.0,30.209205,-97.784344,"(30.20920504, -97.78434414)",2
13,2019911036,THEFT,600,N,04/01/2019 03:02:00 PM,04/01/2019,1502.0,04/01/2019 03:02:00 PM,04/01/2019,1502.0,...,N,04/03/2019,23H,Theft,3129661.0,3129661.0,30.312368,-97.692964,"(30.31236783, -97.69296426)",4


In [11]:
# Looking at the number of incidents associated with each cluster, it seems cluster 0 is the highest crime location

cluster_counts = crime["location_cluster"].value_counts()

# Print the cluster counts
print(cluster_counts)

0    152816
1    107917
4     83675
2     74613
3     41659
Name: location_cluster, dtype: int64


In [12]:
# Looking at what part of Austin that cluster 0 encompasses

# Filter the dataframe for cluster 0
cluster_0 = crime[crime["location_cluster"] == 0]

# Extract longitude and latitude range
longitude_range = (cluster_0["Longitude"].min(), cluster_0["Longitude"].max())
latitude_range = (cluster_0["Latitude"].min(), cluster_0["Latitude"].max())

# Print the longitude and latitude range
print("Longitude range:", longitude_range)
print("Latitude range:", latitude_range)

Longitude range: (-97.78913593, -97.68966993)
Latitude range: (30.14646904, 30.3380483)


In [15]:
# Since we have the longitude and latitudes of each of the incidents, I wanted to look at where on a map were 
# these incidents located, and using folium was perfect for that.

import folium

# Create a map centered on the latitude and longitude range of cluster 0
m = folium.Map(location=[latitude_range[0], longitude_range[0]], zoom_start=12)

# Add markers for each incident in cluster 0, for the first 1000 incidents (showing all 152,000 crashed my MacBook)
count = 0
for index, row in cluster_0.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(m)
    count += 1
    if count == 1000:
        break

# Display the map
m

In [52]:
# Group the dataframe by location_cluster and Highest Offense Description
grouped = crime.groupby("location_cluster")["Highest Offense Description"]

# Iterate over each group and display the top 3 highest offense descriptions
for cluster, group in grouped:
    print(f"Location Cluster {cluster}:")
    top_3_offenses = group.value_counts().head(3)
    print(top_3_offenses)
    print()

Location Cluster 0:
THEFT                    43712
BURGLARY OF VEHICLE      41908
BURGLARY OF RESIDENCE    14174
Name: Highest Offense Description, dtype: int64

Location Cluster 1:
BURGLARY OF VEHICLE     28771
THEFT                   28627
THEFT BY SHOPLIFTING    11048
Name: Highest Offense Description, dtype: int64

Location Cluster 2:
THEFT                    22609
BURGLARY OF VEHICLE      20809
BURGLARY OF RESIDENCE     8043
Name: Highest Offense Description, dtype: int64

Location Cluster 3:
BURGLARY OF VEHICLE     13863
THEFT                   11935
THEFT BY SHOPLIFTING     4564
Name: Highest Offense Description, dtype: int64

Location Cluster 4:
THEFT                    25418
BURGLARY OF VEHICLE      17960
BURGLARY OF RESIDENCE     9428
Name: Highest Offense Description, dtype: int64



In [53]:
# Count the occurrences of each highest offense description overall, strong similarities to clusters as expected

top_offenses = crime["Highest Offense Description"].value_counts().head(3)

# Print the top 3 highest offense descriptions
print(top_offenses)

THEFT                    132301
BURGLARY OF VEHICLE      123311
BURGLARY OF RESIDENCE     45721
Name: Highest Offense Description, dtype: int64


In [206]:
# Showing the most common offense each year along with the number of occurences of that offense
# Clearly, Theft, Stealing Vehicles, and Family related issues are three of Austin's biggest crime related problems

# Group the data by year and highest offense description, and count the occurrences
grouped = crime.groupby(["Year", "Highest Offense Description"]).size().reset_index(name="Count")

# Sort the data by year and occurrence count
grouped_sorted = grouped.sort_values(by=["Year", "Count"], ascending=[True, False])
grouped_sorted.drop_duplicates(subset="Year", keep="first", inplace=True)

# Generate a color map for the highest offense descriptions
offense_descriptions = grouped_sorted["Highest Offense Description"].unique()
color_map = px.colors.qualitative.Dark2[:len(offense_descriptions)]

# Create a bar plot
fig = px.bar(
    grouped_sorted,
    x="Year",
    y="Count",
    color="Highest Offense Description",
    color_discrete_sequence=color_map
)

# Set plot title and labels
fig.update_layout(
    title="Most Common Highest Offense Description Each Year",
    xaxis_title="Year",
    yaxis_title="Occurrences",
    legend_title="Highest Offense Description",
    plot_bgcolor='rgb(17,17,17)',  # Set dark background color
    paper_bgcolor='rgb(17,17,17)',  # Set dark background color
    font=dict(color='white')  # Set text color to white
)

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-90)

# Adjust the legend position
fig.update_layout(legend=dict(orientation="v", yanchor="top", y=0.99, xanchor="left", x=.70))

# Show the figure
fig.show()

In [200]:
# Clearly, crime was steadily decreasing between 2008-2018, but there was a sharp drop in 2019. If this dataset
# included 2020-2022, I would be curious to see if this decline continued, and how COVID affected it.

# Convert the "Occurred Date" column to datetime format
crime["Occurred Date"] = pd.to_datetime(crime["Occurred Date"])

# Extract the year from the "Occurred Date" column
crime["Year"] = crime["Occurred Date"].dt.year

# Group the data by year and count the occurrences
grouped = crime.groupby("Year").size().reset_index(name="Count")

# Create the bar plot
fig = go.Figure(data=[go.Bar(x=grouped["Year"].astype(str), y=grouped["Count"])])

# Set plot title and labels
fig.update_layout(
    title="Total Number of Incidents Each Year",
    xaxis=dict(title="Year"),
    yaxis=dict(title="Total Incidents"),
    plot_bgcolor="rgb(17, 17, 17)",
    paper_bgcolor="rgb(17, 17, 17)",
    font=dict(color="white")
)

# Display the plot
fig.show()

In [35]:
# The growth of the tech field in Austin is something I'm highly interested in as a student looking towards
# working in data science. Lets look at historical data to see how this has changed over time.

occupations = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Occupations.csv')
occupations.head()

,ID Group,Group,ID Subgroup,Subgroup,ID Occupation,Occupation,ID Year,Year,ID State,State,...,Geography,ID Geography,Slug Geography,Median Earnings,Median Earnings Moe,Workforce Prev,Workforce Growth,Median Earnings Prev,Median Earnings Growth,Share
0,0,"Management, Business, Science, & Arts Occupations",0,"Management, Business, & Financial Occupations",0,Management Occupations,2020,2020,04000US48,Texas,...,"Austin, TX",16000US4805000,austin-tx,153702,6137,70602.0,5.967253,148166.0,3.736350,13.298340
1,0,"Management, Business, Science, & Arts Occupations",0,"Management, Business, & Financial Occupations",1,Business & Financial Operations Occupations,2020,2020,04000US48,Texas,...,"Austin, TX",16000US4805000,austin-tx,129948,7039,40619.0,9.520175,127156.0,2.195728,7.907371
2,0,"Management, Business, Science, & Arts Occupations",1,"Computer, Engineering, & Science Occupations",2,Computer & Mathematical Occupations,2020,2020,04000US48,Texas,...,"Austin, TX",16000US4805000,austin-tx,162123,6594,38342.0,7.180116,159578.0,1.594831,7.304622
3,0,"Management, Business, Science, & Arts Occupations",1,"Computer, Engineering, & Science Occupations",3,Architecture & Engineering Occupations,2020,2020,04000US48,Texas,...,"Austin, TX",16000US4805000,austin-tx,164586,8507,16743.0,12.578391,162693.0,1.163541,3.350403
4,0,"Management, Business, Science, & Arts Occupations",1,"Computer, Engineering, & Science Occupations",4,"Life, Physical, & Social Science Occupations",2020,2020,04000US48,Texas,...,"Austin, TX",16000US4805000,austin-tx,104597,16488,6982.0,4.969923,116429.0,-10.162417,1.302727


In [78]:
# Filter the data for subgroup 1 and Austin, TX
subgroup_1_df = occupations[(occupations['ID Subgroup'] == 1) & (occupations['Slug Geography'] == 'austin-tx')]

# Group the data by year and calculate the average median earnings for each year
time_series = subgroup_1_df.groupby('ID Year')['Median Earnings'].mean().round(2)

# Reset the index to convert the result back to a data frame
time_series = time_series.reset_index()
    
time_series.head()

,ID Year,Median Earnings
0,2013,120655.33
1,2014,121689.33
2,2015,128663.33
3,2016,129749.00
4,2017,133549.33


In [79]:
# The growth of median earnings from the tech field is overall positive from year to year, with continous growth 
# from 2013 - 2019, with a sudden drop in 2020 (which is likely due to COVID). The growth of tech is promising 
# in Austin.

# Add a new column for percent growth
time_series['Percent Growth'] = 'na'

# Iterate through the rows and calculate percent growth
for i in range(1, len(time_series)):
    previous_year_earnings = time_series.loc[i - 1, 'Median Earnings']
    current_year_earnings = time_series.loc[i, 'Median Earnings']
    percent_growth = ((current_year_earnings - previous_year_earnings) / previous_year_earnings) * 100
    time_series.loc[i, 'Percent Growth'] = percent_growth

# Display the updated DataFrame
print(time_series)

   ID Year  Median Earnings Percent Growth
0     2013        120655.33             na
1     2014        121689.33       0.856987
2     2015        128663.33       5.730987
3     2016        129749.00       0.843807
4     2017        133549.33       2.928986
5     2018        141797.00       6.175748
6     2019        146233.33       3.128649
7     2020        143768.67       -1.68543


In [234]:
# Now let's explore weather in Austin, Texas

weather = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/austin_weather.csv')
weather.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,30.33,30.27,10,10,7,10,2,16,T,


In [224]:
# Given the amount of numerical data in this dataset, this is perfect for various time series and statistical 
# computations

# Plot the time series to visualize the data, A seasonal trend is clearly evident with Austin data from
# 2013-12-21 to 2017-07-31 (the range of dates in this data set)

# Create the Plotly figure with a dark background
fig = go.Figure()

# Add the average temperature trace
fig.add_trace(go.Scatter(x=weather.index, y=weather['TempAvgF'], mode='lines', name='Average Temperature'))

# Set the layout with a dark background
fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
    title='Average Temperature Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Temperature (°F)'),
    legend=dict(font=dict(color='white'))
)

# Display the plot
fig.show()

In [247]:
# Perform STL decomposition
stl = STL(weather['TempAvgF'], period = 12)
result = stl.fit()

# Extract the components
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Create a Plotly figure with a dark background
fig = go.Figure()

# Add the original data trace
fig.add_trace(go.Scatter(x=weather.index, y=weather['TempAvgF'], mode='lines', name='Original'))

# Add the trend trace
fig.add_trace(go.Scatter(x=weather.index, y=trend, mode='lines', name='Trend'))

# Add the seasonal trace
fig.add_trace(go.Scatter(x=weather.index, y=seasonal, mode='lines', name='Seasonality'))

# Add the residual trace
fig.add_trace(go.Scatter(x=weather.index, y=residual, mode='lines', name='Residuals'))

# Set the layout with a dark background
fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
    title='STL Decomposition of Temperature',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Temperature (°F)'),
    legend=dict(font=dict(color='white'))
)

# Display the plot
fig.show()

In [226]:
# Create an ARIMA model and fit it to the data.

# The coefficients for both the AR(1) and MA(1) terms are significant with p-values of 0.
# Log Likelihood, AIC, and BIC: These values suggest that the model performs reasonably well in terms of goodness 
# of fit.

from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(weather['TempAvgF'], order=(1, 1, 1))
model_fit = model.fit()

# Print model summary
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:               TempAvgF   No. Observations:                 1319
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -4075.486
Date:                Thu, 18 May 2023   AIC                           8156.971
Time:                        18:03:57   BIC                           8172.523
Sample:                             0   HQIC                          8162.803
                               - 1319                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.6408      0.027     23.372      0.000       0.587       0.695
ma.L1         -0.9090      0.016    -58.369      0.000      -0.940      -0.878
sigma2        28.3909      0.835     34.015      0.0

In [227]:
# Create a weather forecast for the weeka fter the data set ends, and compare it to real weather values to see
# if it was accurate.

# Real Weather data From 2017-08-01 to 2017-08-07: 85.42, 82.17, 81.58, 85.58, 86, 86.87, 76.15, 82.68
# Overall, our model's predicitons were within our lower and upper range, and were only a few degrees off each day.
# Our model performed quite well for these predicitions.

# Specify the range of data for model fitting
start_date_fit = '2013-12-21'
end_date_fit = '2021-12-31'
data_for_model = weather.loc[start_date_fit:end_date_fit, 'TempAvgF']

# Fit the ARIMA model
model = ARIMA(data_for_model, order=(1, 1, 1))
model_fit = model.fit()

# Specify the range for the forecast
start_date_forecast = '2023-05-16'
end_date_forecast = '2023-05-22'

# Generate the forecast
forecast = model_fit.get_forecast(steps=7)

# Extract the forecasted values and confidence intervals
forecast_values = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Print the forecasted values
print(forecast_values)
print(forecast_ci)

203    83.0
204    83.0
205    83.0
206    83.0
207    83.0
208    83.0
209    83.0
Name: predicted_mean, dtype: float64
     lower TempAvgF  upper TempAvgF
203       81.040036       84.959964
204       80.228192       85.771808
205       79.605243       86.394757
206       79.080072       86.919928
207       78.617387       87.382613
208       78.199088       87.800912
209       77.814423       88.185577


/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/mlemodel.py:1234: RuntimeWarning:

invalid value encountered in true_divide



In [209]:
# Now that we've explored Austin's weather data, let's see if we can explain Austin's biking data using this.

bike = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/austin_bikeshare_trips.csv')
bike.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,trip_id,year
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,9900082882,2015.0
1,141.0,2:06:04,6,2570.0,South Congress & Academy,10.0,2494.0,2nd & Congress,2016-10-30 02:06:04,Local365,12617682,2016.0
2,578.0,16:28:27,13,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2538.0,Bullock Museum @ Congress & MLK,2016-03-11 16:28:27,Local365,9075366,2016.0
3,555.0,15:12:00,80,2712.0,Toomey Rd @ South Lamar,11.0,2497.0,Capitol Station / Congress & 11th,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,2014.0
4,86.0,15:39:13,25,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,4.0,2707.0,Rainey St @ Cummings,2017-04-16 15:39:13,Walk Up,14468597,2017.0


In [211]:
# Filter the data for the year 2015
df_2015 = bike[bike['year'] == 2015]

# Group the data by month and count the number of checkouts
monthly_checkouts = df_2015.groupby('month')['checkout_time'].count()

# Create the bar graph using Plotly
fig = go.Figure(data=[go.Bar(x=monthly_checkouts.index, y=monthly_checkouts.values)])

# Set the layout with a dark background
fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font=dict(color='white')
)

# Set the axis labels and title
fig.update_xaxes(title_text='Month')
fig.update_yaxes(title_text='Number of Checkouts')
fig.update_layout(title='Number of Bike Checkouts per Month in 2015', title_font_color='white')

# Display the plot
fig.show()

In [215]:
# March and October clearly had the highest number of bike checkouts in 2015 (this holds true for other years as well)
# Lets explore why.

# Convert 'Date' column to datetime format
weather['Date'] = pd.to_datetime(weather['Date'])

# Filter the data for the year 2015
weather_2015 = weather[weather['Date'].dt.year == 2015]

# Convert 'HumidityAvgPercent' and 'TempAvgF' columns to numeric (if necessary)
weather_2015['HumidityAvgPercent'] = pd.to_numeric(weather_2015['HumidityAvgPercent'], errors='coerce')
weather_2015['TempAvgF'] = pd.to_numeric(weather_2015['TempAvgF'], errors='coerce')

# Group the data by month and calculate the average humidity and temperature
monthly_avg_humidity = weather_2015.groupby(weather_2015['Date'].dt.month)['HumidityAvgPercent'].mean()
monthly_avg_temperature = weather_2015.groupby(weather_2015['Date'].dt.month)['TempAvgF'].mean()

# Create a DataFrame to store the monthly averages
monthly_metrics_2015 = pd.DataFrame({'Average Humidity': monthly_avg_humidity, 'Average Temperature': monthly_avg_temperature})

# Display the monthly metrics
monthly_metrics_2015

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_7284/1128105833.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_7284/1128105833.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Average Humidity,Average Temperature
Date,,
1,67.032258,48.935484
2,69.678571,51.607143
3,74.483871,60.483871
4,71.633333,71.433333
5,78.516129,75.419355
6,75.033333,81.333333
7,66.419355,85.838710
8,58.766667,87.645161
9,61.033333,83.566667


In [132]:
# Lets merge the two data sets for easier analysis. We have to create a date column for the bike dataset.

# Convert the 'start_time' column to datetime format (if not already)
bike['start_time'] = pd.to_datetime(bike['start_time'])

# Extract the date from the 'start_time' column and create a new 'date' column
bike['Date'] = bike['start_time'].dt.date

bike.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,trip_id,year,Date
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,9900082882,2015.0,2015-03-19
1,141.0,2:06:04,6,2570.0,South Congress & Academy,10.0,2494.0,2nd & Congress,2016-10-30 02:06:04,Local365,12617682,2016.0,2016-10-30
2,578.0,16:28:27,13,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2538.0,Bullock Museum @ Congress & MLK,2016-03-11 16:28:27,Local365,9075366,2016.0,2016-03-11
3,555.0,15:12:00,80,2712.0,Toomey Rd @ South Lamar,11.0,2497.0,Capitol Station / Congress & 11th,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,2014.0,2014-11-23
4,86.0,15:39:13,25,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,4.0,2707.0,Rainey St @ Cummings,2017-04-16 15:39:13,Walk Up,14468597,2017.0,2017-04-16


In [136]:
# Now we convert both of them to date time and merge

bike['Date'] = pd.to_datetime(bike['Date'])
weather['Date'] = pd.to_datetime(weather['Date'])

merged_data = pd.merge(bike, weather, on='Date', how='inner')
merged_data.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,29.95,29.88,10,8,2,12,5,17,0,
1,62.0,16:12:00,22,2564.0,5th & San Marcos,3.0,2499.0,City Hall / Lavaca & 2nd,2015-03-19 16:12:00,Walk Up,...,29.95,29.88,10,8,2,12,5,17,0,
2,195.0,20:12:00,29,1008.0,Nueces @ 3rd,3.0,2563.0,Davis at Rainey Street,2015-03-19 20:12:00,Walk Up,...,29.95,29.88,10,8,2,12,5,17,0,
3,728.0,15:12:00,31,2539.0,Convention Center / 3rd & Trinity,3.0,2711.0,Barton Springs @ Kinney Ave,2015-03-19 15:12:00,Walk Up,...,29.95,29.88,10,8,2,12,5,17,0,
4,920.0,17:12:00,247,2822.0,East 6th at Robert Martinez,3.0,2575.0,Riverside @ S. Lamar,2015-03-19 17:12:00,Walk Up,...,29.95,29.88,10,8,2,12,5,17,0,


In [190]:
# Displaying the number of rentals vs the average temperature and humidity in 2015

# There doesn't seem to be any obvious trends in the graph below, however the lowest month for humidity has the 
# second highest number of rentals. Additionally, a combination of low to average temperatures and humidities seem
# to cause the highest number of rentals in March.

# There could also very easily be non-weather related explanations for this, such as Spring break and SXSW in March,
# as well as ACL in October.

# Filter the data for the year 2015
df_2015 = merged_data[merged_data['Date'].dt.year == 2015]

# Convert the necessary columns to numeric data type
df_2015['HumidityAvgPercent'] = pd.to_numeric(df_2015['HumidityAvgPercent'], errors='coerce')
df_2015['TempAvgF'] = pd.to_numeric(df_2015['TempAvgF'], errors='coerce')

# Calculate the average rentals by month
monthly_avg_rentals = df_2015.groupby(df_2015['Date'].dt.month)['checkout_time'].count()

# Calculate the average humidity and temperature for each month in 2015
monthly_avg_humidity = df_2015.groupby(df_2015['Date'].dt.month)['HumidityAvgPercent'].mean()
monthly_avg_temperature = df_2015.groupby(df_2015['Date'].dt.month)['TempAvgF'].mean()

# Create a DataFrame to store the monthly metrics
monthly_metrics_2015 = pd.DataFrame({'Average Rentals': monthly_avg_rentals,
                                     'Average Humidity': monthly_avg_humidity,
                                     'Average Temperature': monthly_avg_temperature})

# Create the Plotly figure with a dark background
fig = go.Figure()

# Add the average rentals trace
fig.add_trace(go.Scatter(
    x=monthly_metrics_2015.index,
    y=monthly_metrics_2015['Average Rentals'],
    mode='lines+markers',
    name='Average Rentals',
    marker=dict(color='red')
))

# Create the secondary y-axis for temperature and humidity
fig.update_layout(yaxis2=dict(title='Temperature and Humidity', side='right', overlaying='y', showgrid=False))

# Add the average temperature trace to the secondary y-axis
fig.add_trace(go.Scatter(
    x=monthly_metrics_2015.index,
    y=monthly_metrics_2015['Average Temperature'],
    mode='lines+markers',
    name='Average Temperature',
    marker=dict(color='blue'),
    yaxis='y2'
))

# Add the average humidity trace to the secondary y-axis
fig.add_trace(go.Scatter(
    x=monthly_metrics_2015.index,
    y=monthly_metrics_2015['Average Humidity'],
    mode='lines+markers',
    name='Average Humidity',
    marker=dict(color='green'),
    yaxis='y2'
))

# Set the layout with a dark background
fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
    title='Trends in Bike Rentals, Temperature, and Humidity (2015)',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Average Rentals', color='red', titlefont=dict(color='red')),
    legend=dict(font=dict(color='white'))
)

# Display the plot
fig.show()

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_7284/64599414.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_7284/64599414.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
# Every college student stresses over rising tuition, so let's look at changes in tuition prices 
# over time in Austin, TX.

tuition = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Average Net Price by Sector.csv')

tuition.head()

,ID Sector,Sector,ID Year,Year,Geography,ID Geography,Slug Geography,Spent,Spent ID,Value
0,1,"Public, 4-year or above",2020,2020,"Austin, TX",16000US4805000,austin-tx,State Tuition,1,6729.0
1,2,"Private not-for-profit, 4-year or above",2020,2020,"Austin, TX",16000US4805000,austin-tx,State Tuition,1,22734.5
2,3,"Private for-profit, 4-year or above",2020,2020,"Austin, TX",16000US4805000,austin-tx,State Tuition,1,15404.5
3,1,"Public, 4-year or above",2019,2019,"Austin, TX",16000US4805000,austin-tx,State Tuition,1,6417.0
4,2,"Private not-for-profit, 4-year or above",2019,2019,"Austin, TX",16000US4805000,austin-tx,State Tuition,1,22111.0


In [218]:
# Group by Year and Sector ID and calculate the average tuition price
grouped_df = pd.DataFrame(tuition.groupby(['Year', 'Sector'])['Value'].mean().reset_index())

# Display the grouped DataFrame
print(grouped_df)

'''  Year                                   Sector         Value
1   2013      Private for-profit, 4-year or above   5534.000000
2   2013  Private not-for-profit, 4-year or above   6656.666667
3   2013                  Public, 4-year or above   4268.666667

26  2020      Private for-profit, 4-year or above   5672.333333
27  2020  Private not-for-profit, 4-year or above   8186.500000
28  2020                  Public, 4-year or above   5499.666667
'''

# Looking at the differences between 2013 - 2020 (as well as steadily over time), its quite clear that 
# tuition prices have risen, but not as much as most would expect.

    Year                                   Sector         Value
0   2013               Private for-profit, 2-year   6592.000000
1   2013      Private for-profit, 4-year or above   5534.000000
2   2013  Private not-for-profit, 4-year or above   6656.666667
3   2013                  Public, 4-year or above   4268.666667
4   2014               Private for-profit, 2-year   6857.500000
5   2014      Private for-profit, 4-year or above   4786.000000
6   2014  Private not-for-profit, 4-year or above   6836.666667
7   2014                  Public, 4-year or above   4498.333333
8   2015               Private for-profit, 2-year  13808.000000
9   2015      Private for-profit, 4-year or above   4916.000000
10  2015  Private not-for-profit, 4-year or above   7170.333333
11  2015                  Public, 4-year or above   4694.333333
12  2016               Private for-profit, 2-year  13540.000000
13  2016      Private for-profit, 4-year or above   5158.000000
14  2016  Private not-for-profit, 4-year

'  Year                                   Sector         Value\n1   2013      Private for-profit, 4-year or above   5534.000000\n2   2013  Private not-for-profit, 4-year or above   6656.666667\n3   2013                  Public, 4-year or above   4268.666667\n\n26  2020      Private for-profit, 4-year or above   5672.333333\n27  2020  Private not-for-profit, 4-year or above   8186.500000\n28  2020                  Public, 4-year or above   5499.666667\n'

In [434]:
# Lastly, as someone who has lived and will continue to live in Austin, let's look at housing data and see if we can 
# predict prices through Machine Learning using both numerical and cateogrical variables.

houses = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Zillow_Austin_11-16-22.csv')
# Kept useful columns
houses = houses.drop(['unformattedPrice', 'info3String', 'address', 'addressStreet', 'variableData'], axis = 1)
houses.head()

,price,addressCity,addressState,addressZipcode,beds,baths,area,latitude,longitude,isZillowOwned,badgeInfo,pgapt,sgapt,zestimate,brokerName
0,445000,Austin,TX,78735,3.0,3.0,1802.0,30.269207,-97.86206,False,NaN,ForSale,For Sale (Broker),1189900,Realty Austin
1,1995000,Austin,TX,78757,4.0,4.0,3443.0,30.333755,-97.73414,False,NaN,ForSale,New Construction,2154600,Jimmy Gilmore & Co
2,929900,Austin,TX,78704,2.0,2.0,1318.0,30.252070,-97.76860,False,NaN,ForSale,For Sale (Broker),1048700,Keller Williams Realty
3,6495000,Austin,TX,78703,5.0,6.0,5000.0,30.290514,-97.75361,False,NaN,ForSale,For Sale (Broker),6553400,Douglas Elliman Real Estate
4,365000,Austin,TX,78754,4.0,2.0,2127.0,30.355240,-97.61292,False,NaN,ForSale,For Sale (Broker),421000,Orchard Brokerage


In [435]:
# Making sure our data types are in a useable format

houses.dtypes

price               int64
addressCity        object
addressState       object
addressZipcode      int64
beds              float64
baths             float64
area              float64
latitude          float64
longitude         float64
isZillowOwned        bool
badgeInfo         float64
pgapt              object
sgapt              object
zestimate           int64
brokerName         object
dtype: object

In [436]:
# Create a Data Pipeline for preprocessing data

# create numerical and categorical transformers
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
])

In [437]:
# Creating categorical and numerical columns
categorical_columns = houses.select_dtypes(include=['object', 'bool']).columns
numerical_columns = houses.select_dtypes(include=['int64', 'float64']).columns

# Remove target variable from numerical columns
numerical_columns = numerical_columns.drop('price')

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)])

# Apply the pipeline to the dataset
X = houses.drop('price', axis=1)
y = np.log(houses['price']) #normalize dependent variable 
X_preprocessed = pipeline.fit_transform(X)

In [438]:
# Testing various machine learning models with our data (specifically: Linear Regression, Random Forest, and XGBoost)

# The Linear Regression model performed terribly, while Random Forest and XGBoost performed quite well around an 
# RMSE of .3. This indicates that there could be multicollinearity present in the data, lets see if we can lower these
# RMSE values.

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
for model_name, model in models.items():
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 443351612704.5251

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 500}
Best RMSE for RandomForest: 0.3201938641288851

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
Best RMSE for XGBoost: 0.2986058712051299



In [439]:
# Utilizing Neural Networks

# The test score below shows an R^2 value of roughly .51. This is not terrible, but let's see if we can 
# get it even higher.

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Create an MLPRegressor instance
mlp = MLPRegressor(random_state=42,max_iter=10000, n_iter_no_change = 3,learning_rate_init=0.001)

# Define the parameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(10,), (10,10), (10,10,10), (25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Create the GridSearchCV object
grid_search_mlp = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_mlp.fit(X_train_scaled, y_train)

# Print the best parameters found during the search
print("Best parameters found: ", grid_search_mlp.best_params_)

# Evaluate the model on the test data
best_score = np.sqrt(-1 * grid_search_mlp.best_score_)
print("Test score: ", best_score)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters found:  {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': 25, 'learning_rate': 'constant', 'solver': 'adam'}
Test score:  0.5114274916755802


In [440]:
# Utilizing Principal Component Analysis

pca = PCA()
X_pca_pre = pca.fit_transform(X_preprocessed)

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Choose the number of components based on the explained variance threshold
n_components = np.argmax(cumulative_explained_variance >= 0.95) + 1

pca = PCA(n_components=n_components)
pipeline_pca = Pipeline(steps=
                        [('preprocessor', preprocessor),
                        ('pca', pca)])

X_pca = pipeline_pca.fit_transform(X)

In [441]:
# The RMSE for Linear Regression is significantly better as we expected through PCA, which shows that 
# Dimesionality reduction worked as expected, however the RMSE for the the other two models are worse through PCA

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids_pca = {}
for model_name, model in models.items():
    grids_pca[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids_pca[model_name].fit(X_train_pca, y_train_pca)
    best_params = grids_pca[model_name].best_params_
    best_score = np.sqrt(-1 * grids_pca[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 0.42199085936391656

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}
Best RMSE for RandomForest: 0.3728073334046184

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best RMSE for XGBoost: 0.340376808386979



In [442]:
# Through PCA we have an R^2 value of roughly .49 as can be seen below, worse than earlier, 
# so let's try Feature Engineering.

X_train_scaled_pca = X_train_pca.copy()
X_test_scaled_pca = X_test_pca.copy()

# Create an MLPRegressor instance
mlp = MLPRegressor(random_state=42,max_iter=10000, n_iter_no_change = 3,learning_rate_init=0.001)

# Define the parameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(10,), (10,10), (10,10,10), (25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Create the GridSearchCV object
grid_search_mlp_pca = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_mlp_pca.fit(X_train_scaled_pca, y_train)

# Print the best parameters found during the search
print("Best parameters found: ", grid_search_mlp_pca.best_params_)

# Evaluate the model on the test data
best_score = np.sqrt(-1 * grid_search_mlp_pca.best_score_)
print("Test score: ", best_score)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters found:  {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': 25, 'learning_rate': 'constant', 'solver': 'adam'}
Test score:  0.49207607831276673


In [443]:
from sklearn.metrics import mean_squared_error
for i in grids.keys():
    print (i + ': ' + str(np.sqrt(mean_squared_error(grids[i].predict(X_test), y_test))))

LinearRegression: 3354699719359.0786
RandomForest: 0.38434253271037777
XGBoost: 0.32109674156525686


In [444]:
# Again, PCA improves Linear Regression significantly, but makes the other models worse.

from sklearn.metrics import mean_squared_error
for i in grids.keys():
    print (i + ': ' + str(np.sqrt(mean_squared_error(grids_pca[i].predict(X_test_pca), y_test))))

LinearRegression: 0.43886974258320427
RandomForest: 0.41417300230005666
XGBoost: 0.3533469722622151


In [445]:
print(str(np.sqrt(mean_squared_error(grid_search_mlp.predict(X_test_scaled),y_test))))

0.5501689441021115


In [446]:
# Both of these RMSE's are higher than can be considered good measures of predictors.

print(str(np.sqrt(mean_squared_error(grid_search_mlp_pca.predict(X_test_scaled_pca),y_test))))

0.723589266491158


In [452]:
# Lets see if we can create new variables through feature engineering to improve R^2 and RMSE

houses = pd.read_csv('/Users/adishsundar/Desktop/Austin Data/Zillow_Austin_11-16-22.csv')
houses = houses.drop(['unformattedPrice', 'address', 'info3String'], axis = 1)
houses.head()

,price,addressStreet,addressCity,addressState,addressZipcode,beds,baths,area,latitude,longitude,isZillowOwned,variableData,badgeInfo,pgapt,sgapt,zestimate,brokerName
0,445000,8916 Mountain Shadows Cv APT B,Austin,TX,78735,3.0,3.0,1802.0,30.269207,-97.86206,False,"{'type': 'PRICE_REDUCTION', 'text': '$20,000 (...",NaN,ForSale,For Sale (Broker),1189900,Realty Austin
1,1995000,1701 Alguno Rd,Austin,TX,78757,4.0,4.0,3443.0,30.333755,-97.73414,False,"{'type': 'DAYS_ON', 'text': '19 days on Zillow'}",NaN,ForSale,New Construction,2154600,Jimmy Gilmore & Co
2,929900,1800 Kinney Ave,Austin,TX,78704,2.0,2.0,1318.0,30.252070,-97.76860,False,"{'type': 'OPEN_HOUSE', 'text': 'Open: Sat. 2-4...",NaN,ForSale,For Sale (Broker),1048700,Keller Williams Realty
3,6495000,2407 Pemberton Pl,Austin,TX,78703,5.0,6.0,5000.0,30.290514,-97.75361,False,"{'type': 'PRICE_REDUCTION', 'text': '$500,000 ...",NaN,ForSale,For Sale (Broker),6553400,Douglas Elliman Real Estate
4,365000,11701 Lansdowne Rd,Austin,TX,78754,4.0,2.0,2127.0,30.355240,-97.61292,False,"{'type': '3D_HOME', 'text': '3D Tour'}",NaN,ForSale,For Sale (Broker),421000,Orchard Brokerage


In [453]:
# Feature Engineering

def custom_features(houses):
    df_out = houses.copy()
    df_out['APT/UNIT'] = df_out['addressStreet'].str.contains('APT|UNIT', case=False).astype(bool)
    df_out['variableDataExtracted'] = df_out['variableData'].str.extract(r"'type': '([^']+)'").astype(object)
    df_out['Zipcode_cat'] = df_out['addressZipcode'].astype(object)
    df_out['beds_cat'] = df_out['beds'].astype(object)
    df_out['baths_cat'] = df_out['baths'].astype(object)
    
    return df_out

feature_engineering_transformer = FunctionTransformer(custom_features)

In [454]:
# Identify categorical and numerical columns

new_cols_categorical = pd.Index(['APT/UNIT'])
new_cols_numeric = pd.Index(['Zipcode_cat', 'beds_cat', 'baths_cat'])

In [455]:
# Update categorical and numerical columns

categorical_columns = houses.select_dtypes(include=['object', 'bool']).columns.append(new_cols_categorical)
numerical_columns = houses.select_dtypes(include=['int64', 'float64']).columns.append(new_cols_numeric)

In [456]:
# Remove target variable from numerical columns
numerical_columns = numerical_columns.drop('price')

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline_fe = Pipeline(steps=[
    ('fe', feature_engineering_transformer),
    ('preprocessor', preprocessor),
    ('pca', pca)])

# Apply the pipeline to your dataset
X = houses.drop('price', axis=1)
y = np.log(houses['price'])
X_preprocessed_fe = pipeline_fe.fit_transform(X)

In [457]:
# Split the data into training and testing sets

# Thse RMSE's we get below are around .4 to .3.

X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_preprocessed_fe, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids_fe = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids_fe[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids_fe[model_name].fit(X_train_fe, y_train_fe)
    best_params = grids_fe[model_name].best_params_
    best_score = np.sqrt(-1 * grids_fe[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 0.4222722762078493

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best RMSE for RandomForest: 0.38124451500502277

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best RMSE for XGBoost: 0.33492033628193674



In [458]:
# The R^2 value we get below is around .36, which is worse than earlier.

X_train_scaled_fe = X_train_fe.copy()
X_test_scaled_fe = X_test_fe.copy()

# Create an MLPRegressor instance

mlp = MLPRegressor(random_state=42, max_iter=10000, n_iter_no_change=3)

# Define the parameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(10,), (10, 10), (10, 25)],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'solver': ['adam', 'sgd'],
    'alpha': [.1, .5, 1, 10, 100],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init' : [0.1]
}

# Create the GridSearchCV object
from sklearn.model_selection import GridSearchCV
grid_search_mlp_fe = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_mlp_fe.fit(X_train_scaled_fe, y_train_fe)

# Print the best parameters found during the search
print("Best parameters found: ", grid_search_mlp_fe.best_params_)

# Evaluate the model on the test data
best_score = np.sqrt(-1 * grid_search_mlp_fe.best_score_)
print("Test score: ", best_score)

Fitting 3 folds for each of 270 candidates, totalling 810 fits


/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/U

Best parameters found:  {'activation': 'tanh', 'alpha': 0.5, 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.1, 'solver': 'sgd'}
Test score:  0.3488004747042856


/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


270 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/adishsundar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/adi

In [459]:
for i in grids.keys():
    print (i + ': ' + str(np.sqrt(mean_squared_error(grids_fe[i].predict(X_test_fe), y_test))))

LinearRegression: 0.4264856623470089
RandomForest: 0.41270423685078644
XGBoost: 0.36760208713306164


In [460]:
grids_fe['MLP'] = grid_search_mlp_fe

best_estimators = [(model_name, grid.best_estimator_) for model_name, grid in grids_fe.items()]

# Define the candidate meta-models
meta_models = {
    'MLP': MLPRegressor(random_state=42, max_iter=10000, n_iter_no_change=3, learning_rate_init=0.001),
    'LinearRegression': LinearRegression(),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each meta-model
meta_param_grids = {
    'MLP': {
        'final_estimator__hidden_layer_sizes': [(10,), (10, 10)],
        'final_estimator__activation': ['relu', 'tanh'],
        'final_estimator__solver': ['adam', 'sgd'],
        'final_estimator__alpha': [ 0.001, 0.01, .1, .5],
        'final_estimator__learning_rate': ['constant', 'invscaling', 'adaptive'],
    },
    'LinearRegression': {},
    'XGBoost': {
        'final_estimator__n_estimators': [100, 200, 500],
        'final_estimator__learning_rate': [0.01, 0.1, 0.3],
        'final_estimator__max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the stacking ensemble
best_score = float('inf')
best_model = None

for meta_name, meta_model in meta_models.items():
    print(f'Training and tuning {meta_name} as the meta-model...')
    stacking_regressor = StackingRegressor(estimators=best_estimators, final_estimator=meta_model, cv=cv)
    grid_search = GridSearchCV(estimator=stacking_regressor, param_grid=meta_param_grids[meta_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_fe, y_train_fe)
    best_params = grid_search.best_params_
    best_rmse = np.sqrt(-1 * grid_search.best_score_)
    
    print(f'Best parameters for {meta_name}: {best_params}')
    print(f'Best RMSE for {meta_name}: {best_rmse}\n')
    
    if best_rmse < best_score:
        best_score = best_rmse
        best_model = grid_search

# Evaluate the best stacking ensemble on the test data
y_pred = best_model.predict(X_test_fe)
rmse = np.sqrt(mean_squared_error(y_test_fe, y_pred))
print(f"Best stacking ensemble's RMSE on test data: {rmse}")

# The RMSE's for all 3 models below are roughly around .3 as can be seen below, and on the test data, the Linear
# Regression ensemble's RMSE on test data is the best at around .35

Training and tuning MLP as the meta-model...
Fitting 3 folds for each of 96 candidates, totalling 288 fits
Best parameters for MLP: {'final_estimator__activation': 'relu', 'final_estimator__alpha': 0.5, 'final_estimator__hidden_layer_sizes': (10,), 'final_estimator__learning_rate': 'constant', 'final_estimator__solver': 'adam'}
Best RMSE for MLP: 0.38289971702631415

Training and tuning LinearRegression as the meta-model...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 0.3302178272244467

Training and tuning XGBoost as the meta-model...
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'final_estimator__learning_rate': 0.1, 'final_estimator__max_depth': 3, 'final_estimator__n_estimators': 100}
Best RMSE for XGBoost: 0.34883929442611034

Best stacking ensemble's RMSE on test data: 0.3489063012092765
[CV] END .................................................... t

In [255]:
# This covers everything I wanted to look at with this project. This has by far been my favorite project to work on
# as it gave me a better sense of various aspects of data science like various Machine Learning models, but I also 
# just loved being able to look at Austin through the perspective of data.